<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />

<!--- @wandbcode{intro-colab} -->

# 🏃‍♀️ Quickstart
Use Weights & Biases for machine learning experiment tracking, dataset versioning, and project collaboration.

<img src="http://wandb.me/mini-diagram" width="650" alt="Weights & Biases" />


<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With just a few lines of code,
you'll get rich, interactive, shareable dashboards
like the one in the screenshot below (interactive version
[here](example](https://wandb.ai/stacey/deep-drive?workspace=user-lavanyashukla))).

![](https://i.imgur.com/nHPvS1C.png)


## 🔒 Data & Privacy

We take security very seriously, and our cloud-hosted dashboard uses industry standard best practices for encryption. If you're working with datasets that cannot leave your enterprise cluster, we have [on-prem](https://docs.wandb.com/self-hosted) installations available. 

It's also easy to download all your data and export it to other tools— like custom analysis in a Jupyter notebook. Here's [more on our API](https://docs.wandb.com/library/api).




Start by installing the library and logging in to your free account.

In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
# Log in to your W&B account
wandb.login()

## 👟 Run an experiment
1️⃣. **Start a new run** and pass in hyperparameters to track

2️⃣. **Log metrics** from training or evaluation

3️⃣. **Visualize results** in the dashboard

In [ ]:
import random

# Launch 5 simulated experiments
for run in range(5):
  # 1️⃣ Start a new run to track this script
  wandb.init(
      # Set entity to specify your username or team name
      # ex: entity="carey",
      # Set the project where this run will be logged
      project="basic-intro", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",})
  
  # This simple block simulates a training loop logging metrics
  offset = random.random() / 5
  for ii in range(2, 10):
      acc = 1 - 2 ** -ii - random.random() / ii - offset
      loss = 2 ** -ii + random.random() / ii + offset
      # 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})
      
  # Mark the run as finished
  wandb.finish()

You can find your interactive dashboard by clicking any of the
[project page]()
(for a project-level dashboard)
or [run page]()
(for a run-level dashboard)
links printed by the cell above.

You can also
[interact with W&B dashboards inside Jupyter](http://wandb.me/jupyter-interact-colab).

Here's a dashboard from an autonomous driving project, rendered interactively inside this notebook using the `%wandb` magic:

In [ ]:
%wandb stacey/deep-drive/runs/1wyssjcx -h 720

# 🧠 Example Model
Train a quick model in Keras and track results with W&B. Check our [examples repo](https://github.com/wandb/examples) for scripts and
[colabs](https://github.com/wandb/examples/tree/master/colabs)
using Keras, PyTorch, TensorFlow, Scikit, XGBoost and more.

## 🥕 Simple Keras DNN
Run this model to train a simple MNIST classifier, and click on the project page link to see your results stream in live to a W&B project.



In addition to logged [metrics](https://docs.wandb.ai/ref/app/pages/run-page#charts-tab),
[system information](https://docs.wandb.ai/ref/app/pages/run-page#system-tab),
[hyperparameters](https://docs.wandb.ai/ref/app/pages/run-page#overview-tab), and
[terminal output](https://docs.wandb.ai/ref/app/pages/run-page#logs-tab),
you'll see an [interactive table](https://docs.wandb.ai/guides/data-vis)
with model inputs and outputs.

See [this Report](http://wandb.me/debug-dashboard)
for an example visualization backed by this table --
a model debugging dashboard --
or read more about our Tables feature [here](https://docs.wandb.ai/guides/data-vis/tables-quickstart).

In [ ]:
import random

import numpy as np
import tensorflow as tf
from wandb.keras import WandbCallback

# Simple Keras Model

# Launch 20 experiments, trying different dropout rates
for run in range(20):
  # Start a run, tracking hyperparameters
  wandb.init(
      project="keras-intro",
      # Set entity to specify your username or team name
      # ex: entity="wandb",
      config={
          "layer_1": 512,
          "activation_1": "relu",
          "dropout": random.uniform(0.01, 0.80),
          "layer_2": 10,
          "activation_2": "softmax",
          "optimizer": "sgd",
          "loss": "sparse_categorical_crossentropy",
          "metric": "accuracy",
          "epoch": 6,
          "batch_size": 256
      })
  config = wandb.config

  # Get the data
  mnist = tf.keras.datasets.mnist
  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train / 255.0, x_test / 255.0
  x_train, y_train = x_train[::5], y_train[::5]  # Subset data for a faster demo
  x_test, y_test = x_test[::20], y_test[::20]
  labels = [str(digit) for digit in range(np.max(y_train) + 1)]

  # Build a model
  model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(config.layer_1, activation=config.activation_1),
      tf.keras.layers.Dropout(config.dropout),
      tf.keras.layers.Dense(config.layer_2, activation=config.activation_2)
      ])

  model.compile(optimizer=config.optimizer,
                loss=config.loss,
                metrics=[config.metric]
                )

  # WandbCallback auto-saves all metrics from model.fit(), plus predictions on validation_data
  logging_callback = WandbCallback(log_evaluation=True)

  history = model.fit(x=x_train, y=y_train,
                      epochs=config.epoch,
                      batch_size=config.batch_size,
                      validation_data=(x_test, y_test),
                      callbacks=[logging_callback]
                      )

  # Mark the run as finished
  wandb.finish()

## 📊 Visualize Results

Click on the "[project page](https://docs.wandb.ai/ref/app/pages/project-page)"
link above to see your live results.



## 🎨 Example Gallery

See examples of projects tracked and visualized with W&B in our gallery, [Fully Connected→](https://app.wandb.ai/gallery)

# 🏙️ Community

Join a community of ML practitioners in our
[Discourse forum→](http://wandb.me/and-you)

# 📏 Best Practices

1. **Projects**: Log multiple runs to a project to compare them. `wandb.init(project="project-name")`
2. **Groups**: For multiple processes or cross validation folds, log each process as a run and group them together. `wandb.init(group='experiment-1')`
3. **Tags**: Add tags to track your current baseline or production model.
4. **Notes**: Type notes in the table to track the changes between runs.
5. **Reports**: Take quick notes on progress to share with colleagues and make dashboards and snapshots of your ML projects.



### 🤓 Advanced Setup

1. [Environment variables](https://docs.wandb.com/library/environment-variables): Set API keys in environment variables so you can run training on a managed cluster.
2. [Offline mode](https://docs.wandb.com/library/technical-faq#can-i-run-wandb-offline): Use `dryrun` mode to train offline and sync results later.
3. [On-prem](https://docs.wandb.com/self-hosted): Install W&B in a private cloud or air-gapped servers in your own infrastructure. We have local installations for everyone from academics to enterprise teams.
4. [Sweeps](http://wandb.me/sweeps-colab): Set up hyperparameter search quickly with our lightweight tool for tuning.
5. [Artifacts](http://wandb.me/artifacts-colab): Track and version models and datasets in a streamlined way that automatically picks up your pipeline steps as you train models.
6. [Tables](http://wandb.me/dsviz-nature-colab): Log, query, and analyze tabular data. Understand your datasets, visualize model predictions, and share insights in a central dashboard.